#此程序最后被*2400的程序所取代，原因是此程序的筛选标准太严格，只能筛选出42个站点。对应的输出文件为df_final_2.csv

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import glob
import datetime
import pandas as pd

data_path='/home/liuli/GTS_GFS_MOS/domestic_hourly_data/df_all.csv'
file0= pd.read_csv(data_path,sep=',')
df0=pd.DataFrame(file0)

In [2]:
#print(df_all.info())
df_all=df0.drop_duplicates(subset = ['time','sta'],keep='last',inplace=False) #去掉重复的行
print(df_all)

                time       sta      ht    lat     lon       pr1      pr24  \
0         2018010100     23471    54.0  60.92   76.60  999999.0  999999.0   
1         2018010100     23552    27.0  64.92   77.82  999999.0  999999.0   
2         2018010100     23578    30.0  64.23   87.57  999999.0  999999.0   
3         2018010100     23589   105.0  64.20   93.78  999999.0  999999.0   
4         2018010100     23656    67.0  63.38   78.32  999999.0  999999.0   
5         2018010100     23657   131.0  63.12   75.28  999999.0  999999.0   
6         2018010100     23662    34.0  63.98   82.08  999999.0  999999.0   
7         2018010100     23678    45.0  63.15   87.97  999999.0  999999.0   
8         2018010100     23699   230.7  62.71  101.11  999999.0  999999.0   
9         2018010100     23741    85.0  62.75   71.67  999999.0  999999.0   
10        2018010100     23748    67.0  62.23   74.48  999999.0  999999.0   
11        2018010100     23758    68.0  62.15   77.32  999999.0  999999.0   

In [3]:
#1.先把大df中某个站点的数据挑选出来
start='2018-01-01 00:00:00'
end_of_db = '2019-01-01 00:00:00'
date_index = pd.date_range(start, end_of_db, freq = '1h')
date_df = pd.DataFrame(date_index,columns=['time'])


#把df_all的时间转换成datatime
df_all['time']=pd.to_datetime(df_all['time'],format="%Y%m%d%H")
#df_all['time'] = df_all['time'].apply(lambda x : datetime.datetime.strptime(x,'%Y%m%d%H'))
#print(df_all['time'].dtype)

#print(type(df_all['time'].dt.hour[0]))
df_qc=df_all.drop_duplicates(subset = ['sta'],keep='last',inplace=False) #申明一个不重复站点的df
print(df_qc)

/home/liuli/anaconda3/envs/MOS/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


                        time       sta      ht    lat     lon       pr1  \
21226    2018-01-01 12:00:00     A8VV3  9999.0  13.30  109.90  999999.0   
55488    2018-01-02 12:00:00     DEDG2  9999.0   4.10   92.30  999999.0   
76655    2018-01-03 03:00:00      7XJX  9999.0  29.10  129.80  999999.0   
81182    2018-01-03 07:00:00    9V9130  9999.0  12.60  112.90  999999.0   
110863   2018-01-04 05:00:00      HONU  9999.0  36.80  143.90  999999.0   
149205   2018-01-05 06:00:00     DGQD2  9999.0   5.90   89.40  999999.0   
174941   2018-01-06 01:00:00    9V5332  9999.0  37.10  143.60  999999.0   
209009   2018-01-07 00:00:00      3FOB  9999.0   5.30   91.50  999999.0   
242942   2018-01-08 00:00:00      CQYN  9999.0  50.60  178.30  999999.0   
272632   2018-01-08 22:00:00    9V3440  9999.0  54.40  176.40  999999.0   
281359   2018-01-09 05:00:00      WAZV  9999.0  46.20  151.30  999999.0   
281366   2018-01-09 05:00:00      WKPM  9999.0  25.40   80.00  999999.0   
319323   2018-01-10 07:00

In [ ]:
#df2=df_all[df_all['sta'].isin(['53603'])]
#print(df2)
#for i in df_qc.sta:  #站点不重复
#    #print(i)
#    #print(type(i))
#    df2=df_all[df_all['sta'].isin([i])] #先挑出这个站号的行 
#    df2=df2.sort_values(by='time')#把这个站点按照时间顺序排列    
#    df2=df2.reset_index(drop=True)
#    if len(df2)>4000:
#        print(df2)

In [ ]:
############开始循环读取每个站点的数据，并判断此站点的有效数据量
df_final=pd.DataFrame(columns=['time', 'sta', 'ht', 'lat', 'lon', 'pr1', 'pr24', 'pr6', 'ps', 'psl', 't2m', 'td', 'wdir', 'wspd'])

for i in df_qc.sta:  #站点不重复
    
    #print(type(i))
    df2=df_all[df_all['sta'].isin([i])] #先挑出这个站号的行
    if len(df2)>=4000: ####先判断此站点的总时间长度是否达到7000（根据24*365=8760的50%有效数据）
        print(i)
        df2=df2.sort_values(by='time')#把这个站点按照时间顺序排列    
        df2=df2.reset_index(drop=True)
        #print(df2)         
        # merge日期补充完整数据
        df2 = pd.merge(date_df,df2, on=['time'], how='left')
        df2['sta'].fillna(i,inplace=True)
        #df2['sta']=df2['sta'].replace('NaN',i)
        #判断03时，处理为3小时累计降水
        for index, item in enumerate(df2['time'].values):
            #从第三行开始，把1h降水量累加成为3h降水量
            if index>=2 and df2['time'].dt.hour[index]%3==0 and df2['pr1'].values[index]<1000.0:
#            if index>=2 and df2['time'].dt.hour[index]%3==0:                
#                print(index,item,df2['sta'].values[index],df2['pr1'].values[index])
                if df2['pr1'].values[index-1]<1000.0 and df2['pr1'].values[index-2]<1000.0:            
                    df2['pr1'].values[index]=df2['pr1'].values[index]+df2['pr1'].values[index-1]+df2['pr1'].values[index-2]
                    print(index,item,df2['sta'].values[index],df2['pr1'].values[index])
                    
                else:
                    df2['pr1'].values[index]=999999.0
                    print(index,item,df2['sta'].values[index],df2['pr1'].values[index])
        df_final=pd.concat([df_final,df2],ignore_index=True)            
    #print(df2)
    #print(df2.info())
    del df2

#这种根据某一列数值，赋值另一列适合比较简单的场景：print(df_all.loc[(float(df_all.time[-2:])%3 ==0), 'pr1'] )
df_final

47625


In [ ]:
df_final.to_csv('./df_final_2.csv', sep=',', header=True, index=False)